# Lab | Tools prompting

**Replace the existing two tools decorators, by creating 3 new ones and adjust the prompts accordingly**

### How to add ad-hoc tool calling capability to LLMs and Chat Models

:::{.callout-caution}

Some models have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling. Please see the [how to use a chat model to call tools](/docs/how_to/tool_calling) guide for more information.

In this guide, we'll see how to add **ad-hoc** tool calling support to a chat model. This is an alternative method to invoke tools if you're using a model that does not natively support tool calling.

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools. Here's a diagram of the logic:

<br>

![chain](https://education-team-2020.s3.eu-west-1.amazonaws.com/ai-eng/tool_chain.svg)

## Setup

We'll need to install the following packages:

In [2]:
#%pip install --upgrade --quiet langchain langchain-community

If you'd like to use LangSmith, uncomment the below:

In [26]:
import getpass
import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

You can select any of the given models for this how-to guide. Keep in mind that most of these models already [support native tool calling](/docs/integrations/chat/), so using the prompting strategy shown here doesn't make sense for these models, and instead you should follow the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

To illustrate the idea, we'll use `phi3` via Ollama, which does **NOT** have native support for tool calling. If you'd like to use `Ollama` as well follow [these instructions](/docs/integrations/chat/ollama/).

In [2]:
from langchain_community.llms import Ollama

from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2",
    temperature=0,
    # other params...
)

In [3]:
from langchain_core.messages import AIMessage

messages = [
    (
        "system",
        "extract the entities from the following text:.",
    ),
    ("human", "TSIA, SIA, the U.S. Chamber of Commerce, the Global Business Alliance, SEMI, ITIC, and the U.S. Taiwan Business Council"),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='Here are the extracted entities:\n\n1. TSIA\n2. SIA\n3. U.S. Chamber of Commerce\n4. Global Business Alliance\n5. SEMI\n6. ITIC\n7. U.S. Taiwan Business Council', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-10-24T13:51:29.480156Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2819702750, 'load_duration': 2091200667, 'prompt_eval_count': 66, 'prompt_eval_duration': 114605000, 'eval_count': 50, 'eval_duration': 605769000}, id='run-19d7fce7-e29d-4ba9-9aed-eeb6d6b454e1-0', usage_metadata={'input_tokens': 66, 'output_tokens': 50, 'total_tokens': 116})

## Create a tool

First, let's create an `add` and `multiply` tools. For more information on creating custom tools, please see [this guide](/docs/how_to/custom_tools).

In [33]:
from langchain_core.tools import tool
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun

@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y

@tool
def wikidata_entires(x: str) -> str:
    """lookup entires on wikipendia"""
    wiki_entries = WikidataQueryRun(api_wrapper=WikidataAPIWrapper())
    wiki_entries_result = wiki_entries(x)
    return wiki_entries_result

tools = [multiply, add,wikidata_entires]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
multiply
Multiply two numbers together.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
add
Add two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}
--
wikidata_entires
lookup entires on wikipendia
{'x': {'title': 'X', 'type': 'string'}}


In [36]:
#multiply.invoke({"x": 4, "y": 5})

wikidata_entires.invoke("U.S. Chamber of Commerce?")


'Result Q7863119:\nLabel: U.S. Chamber of Commerce Building\nDescription: headquarters building of the United States Chamber of Commerce, in Washington, D.C.\nAliases: United States Chamber of Commerce Building, US Chamber of Commerce Building, Chamber of Commerce of the United States of America Building\n\nResult Q16956993:\nLabel: U.S. Chamber of Commerce Foundation\nDescription: nonprofit organization affiliated with the United States Chamber of Commerce\nAliases: U.S. Chamber Foundation, National Chamber Foundation'

In [38]:

wikidata = WikidataQueryRun(api_wrapper=WikidataAPIWrapper())

print(wikidata_entires.run("Q106411969"))

Result Q106411969:
Label: Taiwan Semiconductor Industry Association
Description: organization in Hsinchu, Taiwan
Aliases: TSIA


## Creating our prompt

We'll want to write a prompt that specifies the tools the model has access to, the arguments to those tools, and the desired output format of the model. In this case we'll instruct it to output a JSON blob of the form `{"name": "...", "arguments": {...}}`.

In [43]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

multiply(x: float, y: float) -> float - Multiply two numbers together.
add(x: int, y: int) -> int - Add two numbers.
wikidata_entires(x: str) -> str - lookup entires on wikipendia


In [44]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [45]:
chain = prompt | llm
message = chain.invoke({"input": "what's 3 plus 1132"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

{
    "name": "add",
    "arguments": {
        "x": 3,
        "y": 1132
    }
}


In [46]:
chain = prompt | llm
message = chain.invoke({"input": "Business Alliance Council"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

Here's the response in JSON format:

```
{
  "name": "wikidata_entires",
  "arguments": {
    "x": "U.S. Chamber of Commerce"
  }
}
```

This indicates that the user requested information on the U.S. Chamber of Commerce, and the tool to use is `wikidata_entires`. The argument passed to this tool is the string `"U.S. Chamber of Commerce"`, which will be used to look up the relevant Wikipedia entry.


## Adding an output parser

We'll use the `JsonOutputParser` for parsing our models output to JSON.

In [47]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | llm | JsonOutputParser()
chain.invoke({"input": "who is the Business Alliance Council"})

{'name': 'wikidata_entires', 'arguments': {'x': 'Business Alliance Council'}}

>:::{.callout-important}

🎉 Amazing! 🎉 We now instructed our model on how to **request** that a tool be invoked.

Now, let's create some logic to actually run the tool!
:::

## Invoking the tool 🏃

Now that the model can request that a tool be invoked, we need to write a function that can actually invoke 
the tool.

The function will select the appropriate tool by name, and pass to it the arguments chosen by the model.

In [48]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

Let's test this out 🧪!

In [49]:
invoke_tool({"name": "multiply", "arguments": {"x": 3, "y": 5}})

15.0

## Let's put it together

Let's put it together into a chain that creates a calculator with add and multiplication capabilities.

In [55]:
chain = prompt | llm | JsonOutputParser() | invoke_tool
chain.invoke({"input": "what is the U.S chamber of commerce"})

'Result Q7863119:\nLabel: U.S. Chamber of Commerce Building\nDescription: headquarters building of the United States Chamber of Commerce, in Washington, D.C.\nAliases: United States Chamber of Commerce Building, US Chamber of Commerce Building, Chamber of Commerce of the United States of America Building\n\nResult Q16956993:\nLabel: U.S. Chamber of Commerce Foundation\nDescription: nonprofit organization affiliated with the United States Chamber of Commerce\nAliases: U.S. Chamber Foundation, National Chamber Foundation'

## Returning tool inputs

It can be helpful to return not only tool outputs but also tool inputs. We can easily do this with LCEL by `RunnablePassthrough.assign`-ing the tool output. This will take whatever the input is to the RunnablePassrthrough components (assumed to be a dictionary) and add a key to it while still passing through everything that's currently in the input:

In [51]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | llm | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "Who is Alen Turing"})

{'name': 'wikidata_entires',
 'arguments': {'x': 'Alan Turing'},
 'output': 'Result Q7251:\nLabel: Alan Turing\nDescription: English computer scientist (1912–1954)\nAliases: Alan M. Turing, Alan Mathieson Turing, Turing, Alan Mathison Turing\n\nResult Q28846012:\nLabel: Alan Turing\nDescription: fictional analogon of Alan Turing (1912-1954)\nAliases: Alan Mathison Turing'}

## What's next?

This how-to guide shows the "happy path" when the model correctly outputs all the required tool information.

In reality, if you're using more complex tools, you will start encountering errors from the model, especially for models that have not been fine tuned for tool calling and for less capable models.

You will need to be prepared to add strategies to improve the output from the model; e.g.,

1. Provide few shot examples.
2. Add error handling (e.g., catch the exception and feed it back to the LLM to ask it to correct its previous output).